# Diffusion process

<img src='https://drive.google.com/uc?id=1aMLLAiDU4d_2C77asMCYHpwOkz1jrc__' width="1000"/>

## Forward diffusion process
The forward diffusion process is a Markovian process that iteratively adds Gaussian noise to a data point over 𝑇 iterations. Each step in the forward direction is given by

$$
q(x_t | x_{t-1}) = \mathcal{N}(x_t; \sqrt{1 - \beta_t} x_{t-1}, \beta_tI)
$$

where $\beta_t$ is a Gaussian noise with variance at timestep $t$

At each timestep $ t $, the parameters that define the distribution of image $x_t$ are set as:
   - Mean: $\sqrt{1 - \beta_t} x_{t-1}$
   - Covariance: $( \beta_t I )$



Given raw data $x_0$, sampling of $x_t$ is expressed as a single step in a closed form:


$$
q(x_t | x_0) = \mathcal{N}(x_t; \sqrt{\hat{\alpha}_t} x_0, (1 - \hat{\alpha}_t))
$$
where $\alpha_t = 1 - \beta_t$ and $\hat{\alpha}_t = \prod_{s=1}^t (1 - \beta_s)$. Therefore, $x_t$ can be expressed as a linear combination of $x_0$ and $\epsilon$ using the reparameterization trick:


Reparameterization Trick:
$$
\mathcal{N}(\mu, \sigma^2) = \mu + \sigma \cdot \epsilon
$$

$$
x_t = \sqrt{\hat{\alpha}_t} x_0 + \sqrt{1 - \hat{\alpha}_t} \epsilon
$$

where $\epsilon \sim \mathcal{N}(0, I)$ and the dimension is the same with $x$.


In [ ]:
  # Forward diffusion process
  def diffuse(self, x, t, noise=None):
      if noise is None:
          noise = torch.randn_like(x)
      alpha_cumprod = self._alpha_cumprod(t).view(t.size(0), *[1 for _ in x.shape[1:]])
      return alpha_cumprod.sqrt() * x + (1 - alpha_cumprod).sqrt() * noise, noise

# Reverse diffusion process
The denoising process is the opposite of the noise addition process. A noise sample is sampled from the standard normal distribution and then gradually denoised to obtain a sample in the data distribution. As the reverse of the forward process $q(x_{t-1} | x_t)$ is intractable, DDPM learns parameterized Gaussian transitions $p_\theta(x_{t-1} | x_t)$. Therefore, in the denoising process, the neural network predicts the parameters of the Gaussian distribution with learned mean $\mu_\theta$ and fixed variance $\beta_t I$.

$$
p_\theta(x_{t-1} | x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \beta_t I)
$$

$$
x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1-\alpha_t}{\sqrt{1-\hat\alpha_t}} \epsilon \theta(x_t, t) \right) + \sqrt{\beta_t} \mathcal{Z}
$$

In [ ]:
class DDPMSampler(SimpleSampler):
    def step(self, x, t, t_prev, noise):
        alpha_cumprod = self.diffuzz._alpha_cumprod(t).view(t.size(0), *[1 for _ in x.shape[1:]])
        alpha_cumprod_prev = self.diffuzz._alpha_cumprod(t_prev).view(t_prev.size(0), *[1 for _ in x.shape[1:]])
        alpha = (alpha_cumprod / alpha_cumprod_prev)

        mu = (1.0 / alpha).sqrt() * (x - (1 - alpha) * noise / (1 - alpha_cumprod).sqrt())
        std = ((1 - alpha) * (1. - alpha_cumprod_prev) / (1. - alpha_cumprod)).sqrt() * torch.randn_like(mu)
        return mu + std * (t_prev != 0).float().view(t_prev.size(0), *[1 for _ in x.shape[1:]])

# Sampling process


<img src='https://drive.google.com/uc?id=1BtdAwtwvJaYAUgvpLxybom0Ok3ZzM2pv' width="1000"/>

In [ ]:
  # Sampling from the diffusion model
  def sample(self, model, background, shape, mask=None, t_start=1.0, t_end=0.0, timesteps=300, x_init=None, sampler='ddpm'):
      r_range = torch.linspace(t_start, t_end, timesteps + 1)[:, None].expand(-1, shape[0] if x_init is None else x_init.size(0)).to(self.device)
      if isinstance(sampler, str):
          if sampler in sampler_dict:
              sampler = sampler_dict[sampler](self)
          else:
              raise ValueError(
                  f"If sampler is a string it must be one of the supported samplers: {list(sampler_dict.keys())}")
      elif issubclass(sampler, SimpleSampler):
          sampler = sampler(self)
      else:
          raise ValueError("Sampler should be either a string or a SimpleSampler object.")
      preds = []
      x = sampler.init_x(shape)

      # Iterate over timesteps and apply diffusion model
      for i in tqdm(range(0, timesteps), desc="Sampling Progress"):
          if mask is not None:
              x_renoised, _ = self.diffuse(x_init, r_range[i])
              x = x * mask + x_renoised * (1 - mask)

          pred_noise = model(torch.cat((x, background), dim=1), r_range[i])
          x = self.undiffuse(x, r_range[i], r_range[i + 1], pred_noise, sampler=sampler)
          preds.append(x)
      return preds

References

https://jalammar.github.io/illustrated-stable-diffusion/

https://github.com/dome272/Diffusion-Models-pytorch/tree/main